In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
file_path = "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/results/base_paper_single_horizon/Optimal Capacity_comparison.csv"
data = pd.read_csv(file_path)
data.head()

In [ ]:
carriers = pd.read_csv(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/results/base_paper_single_horizon/carriers.csv"
)
carriers = carriers.set_index("nice_name").color
carriers

In [ ]:
# Combine 'Battery Storage' and '4Hr_Battery_Storage' rows
data['nice_name'] = data['nice_name'].replace({'4Hr_Battery_Storage': 'Battery Storage'})

# Group by the necessary columns and sum the statistics
data = data.groupby(['nice_name', 'Scenario', 'horizon', 'scenario_name', 'trans_expansion'], as_index=False).sum()

In [ ]:
# Remove 'Ac' from data
data = data[data['nice_name'] != 'Ac']

In [ ]:
# Preprocessing the trans_expansion column to extract numerical values
# Assuming '0x', '1x', etc., represent numeric multipliers like 1, 2, etc.
data["trans_expansion_numeric"] = data["trans_expansion"].str.extract(r"(\d+\.?\d*)").astype(float)

# Checking for unique scenario names and valid numeric values
unique_scenarios = data["scenario_name"].unique()
trans_expansion_check = data["trans_expansion_numeric"].unique()

unique_scenarios, trans_expansion_check

In [ ]:
# Filter out rows with NaN in the 'trans_expansion_numeric' column
data_cleaned = data.dropna(subset=["trans_expansion_numeric"])

# Group the data by scenario_name for plotting
grouped_data = data_cleaned.groupby("scenario_name")

In [ ]:
sys_cost = pd.read_csv(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/results/base_paper_single_horizon/System Costs_comparison.csv"
).round(3)
# sys_cost.Scenario.map(
trans_exp_dict = data_cleaned.set_index("Scenario")["trans_expansion_numeric"].to_dict()
scenario_dict = data_cleaned.set_index("Scenario")["scenario_name"].to_dict()
sys_cost["trans_expansion_numeric"] = sys_cost.Scenario.map(trans_exp_dict)
sys_cost["scenario_name"] = sys_cost.Scenario.map(scenario_dict)
sys_cost.dropna(subset=["trans_expansion_numeric"], inplace=True)
sys_cost

In [ ]:
# Initialize the plot again
font_size = 30
fig, axs = plt.subplots(1, len(unique_scenarios), figsize=(12, 6), sharey=True)
fig.subplots_adjust(wspace=0.1)

# Plot each scenario as a subplot
for ax, (scenario, scenario_data) in zip(axs, grouped_data):
    # Sort data by trans_expansion_numeric for consistent plotting
    scenario_data = scenario_data.sort_values("trans_expansion_numeric")
    cost_scenario_data = sys_cost[sys_cost["scenario_name"] == scenario]

    # Create a twin y-axis to plot cost_scenario_data
    ax2 = ax.twinx()

    # Group by nice_name to plot separate lines for each
    for nice_name, group_data in scenario_data.groupby("nice_name"):
        ax.plot(
            group_data["trans_expansion_numeric"],
            group_data["statistics"],
            marker="o",
            label=nice_name,
            color=carriers[nice_name],
        )

    # Plot cost_scenario_data on the secondary y-axis
    ax2.plot(
        cost_scenario_data["trans_expansion_numeric"],
        cost_scenario_data["statistics"],
        marker="x",
        linestyle="--",
        color="black",
        label="System Cost",
    )

    # Set labels and title
    ax.set_title(f"{scenario}", fontsize=font_size)
    ax.set_xlabel("Transmission Expansion Factor", fontsize=font_size)
    ax.set_xscale("linear")  # Adjust scaling if necessary
    ax.grid(True)

    # Set secondary y-axis label
    # ax2.set_ylabel("System Cost", fontsize=font_size)

    # Set y-axis limits for both primary and secondary y-axes
    ax.set_ylim(50, 1000)
    ax2.set_ylim(80, 220)

# Set the shared y-axis label
axs[0].set_ylabel("Optimal Capacity [GW]", fontsize=font_size)

# Increase font size of x and y axis
for ax in axs:
    ax.tick_params(axis="x", labelsize=font_size)
    ax.tick_params(axis="y", labelsize=font_size)
    for line in ax.get_lines():
        line.set_linewidth(4)

ax2.tick_params(axis="y", labelsize=font_size)
ax2.set_ylabel("System Cost [$B/yr]", fontsize=font_size)

# Add legend to the first subplot for all lines (common legend)
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, title="", loc="upper center", bbox_to_anchor=(0.5, -0.05), fontsize=font_size, ncol=2)

# Show the plot
plt.tight_layout()
plt.show()